In [30]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd

# Define the LSTM model
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

    
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [31]:

# Set random seed for reproducibility
torch.manual_seed(123)

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define hyperparameters
input_size = 1  # Number of input features (number of previous days' data)
hidden_size = 32  # Number of LSTM units
num_layers = 3  # Number of LSTM layers
output_size = 1  # Number of output features (predicted number of infected people)
num_epochs = 100  # Number of training epochs
learning_rate = 0.001  # Learning rate


model = LSTMModel(input_size, hidden_size, num_layers, output_size).to(device)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


# Generate dummy data (replace with your actual data)

okuzeni_all = pd.read_csv('okuzeni.csv')
NAMES = [ "ljubljana", "maribor", "kranj", "koper", "celje", "novo_mesto", "velenje", "nova_gorica", "krško", "ptuj", "murska_sobota", "slovenj_gradec"]
okuzeni = np.array(okuzeni_all[NAMES].values)
#okuzeni = torch.tensor(np.array(okuzeni_all[NAMES].values)).type(torch.Tensor)
data = okuzeni[:,0].reshape(-1, 1)
#print(data.reshape(-1, 1))
#data = np.random.rand(1000) * 1000
#data = data.reshape(-1, 1)
#print(data)
# Split the data into training and testing sets

train_size = int(0.7 * len(data))
train_data = data[:train_size]
test_data = data[train_size:]

# Convert data to tensors
train_data_tensor = torch.from_numpy(train_data).float().to(device)
test_data_tensor = torch.from_numpy(test_data).float().to(device)

# Prepare the input and target data
def prepare_data(data, input_size):
    X, Y = [], []
    for i in range(len(data) - input_size):
        X.append(data[i:i+input_size])
        Y.append(data[i+input_size])
    return torch.stack(X).to(device), torch.stack(Y).to(device)

# Prepare the training and testing data
train_input, train_target = prepare_data(train_data_tensor, input_size)
test_input, test_target = prepare_data(test_data_tensor, input_size)


In [33]:

# Training loop
for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    train_output = model(train_input)
    loss = criterion(train_output, train_target)
    loss.backward()
    optimizer.step()

    # Print training loss every 10 epochs
    if (epoch+1) % 10 == 0:
        print(f"Epoch: {epoch+1}/{num_epochs}, Loss: {loss.item()}")

# Evaluation
model.eval()
with torch.no_grad():
    test_output = model(test_input)
    test_loss = criterion(test_output, test_target)
    print(f"Test Loss: {test_loss.item()}")

# Convert tensors to numpy arrays
#train_output = train


Epoch: 10/100, Loss: 25573772.0
Epoch: 20/100, Loss: 25570412.0
Epoch: 30/100, Loss: 25567588.0
Epoch: 40/100, Loss: 25565038.0
Epoch: 50/100, Loss: 25562638.0
Epoch: 60/100, Loss: 25560356.0
Epoch: 70/100, Loss: 25558156.0
Epoch: 80/100, Loss: 25556028.0
Epoch: 90/100, Loss: 25553952.0
Epoch: 100/100, Loss: 25551932.0
Test Loss: 5812937.5
